# State Machines

[A Machine State of Mind](http://vaidehijoshi.github.io/blog/2015/03/17/a-machine-state-of-mind-part-1-understanding-state-machines/)

[DOT Graphviz language](https://graphs.grevian.org/example)

In [ ]:
import graphviz

In [ ]:
!cat topics-3.dot

In [ ]:
graphviz.Source.from_file('topics-3.dot')

In [ ]:
from transitions import Machine
from dataclasses import dataclass

In [ ]:
@dataclass
class Topic:
    
    name: str
        
    def __post_init__(self):
        self.presenter = None
        self.machine = Machine(
            model=self, 
            states=['proposed', 'scheduled', 'completed', ], 
        )
        

Warning!  Do not create a Machine pointing to a class... it almost works, but then does not (for instance, on callbacks)

In [ ]:
t1 = Topic('Block programming')

In [ ]:
t1

Now we have a built-in state

In [ ]:
t1.state

And built-in methods to change the state

In [ ]:
t1.to_proposed()

In [ ]:
t1.state

In [ ]:
t1.to_scheduled()
t1.state

Oops, let's set a real initial state

In [ ]:
@dataclass
class Topic:
    
    name: str
    
    def __post_init__(self):
        self.presenter = None
        self.machine = Machine(
            model=self, 
            states=['proposed', 'scheduled', 'completed', ], 
            initial = 'proposed',
        )
t2 = Topic('time zones')

In [ ]:
t2.state

But these aren't proper transitions

In [ ]:
t2.to_completed()
t2.state

## Transitions



In [ ]:
@dataclass
class Topic:
    
    name: str
        
    def __post_init__(self):
        self.presenter = None
        self.machine = Machine(
            model=self, 
            states=['proposed', 'scheduled', 'completed', ], 
            initial='proposed',
            transitions=[
                {'trigger': 'schedule', 'source': 'proposed', 'dest': 'scheduled'},
                {'trigger': 'complete', 'source': 'scheduled', 'dest': 'completed'},        
            ])  
t3 = Topic('Immutability')

In [ ]:
t3.state

In [ ]:
t3.complete()

In [ ]:
t3.schedule()

In [ ]:
t3.state

In [ ]:
t3.complete()

In [ ]:
t3.state

Don't use the `to_` methods.  They're cheating.  You can even disable them.

## Callbacks

In [ ]:
@dataclass
class Topic:
    
    name: str
        
    def __post_init__(self):
        self.presenter = None
        self.machine = Machine(
            model=self, 
            states=['proposed', 'scheduled', 'completed', ], 
            initial='proposed',
            transitions=[
                {'trigger': 'schedule', 'source': 'proposed', 'dest': 'scheduled'},
                {'trigger': 'complete', 'source': 'scheduled', 'dest': 'completed'},        
            ])  
        
    def on_enter_scheduled(self):
        print('Update the website!')
        
t4 = Topic('FOSS contribution')

In [ ]:
t4.schedule()

## Conditional transitions

In [ ]:
@dataclass
class Topic:
    
    name: str
        
    def has_presenter(self):
        return self.presenter is not None
        
    def __post_init__(self):
        self.presenter = None
        self.machine = Machine(
            model=self, 
            states=['proposed', 'scheduled', 'completed', ], 
            initial='proposed',
            transitions=[
                {'trigger': 'schedule', 'source': 'proposed', 'dest': 'scheduled',
                'conditions': ['has_presenter', ]},
                {'trigger': 'complete', 'source': 'scheduled', 'dest': 'completed'},        
            ])  
t5 = Topic('Fluid Interfaces')

In [ ]:
t5.schedule()

In [ ]:
t5.state

In [ ]:
t5.presenter = 'Ken'

In [ ]:
t5.schedule()

In [ ]:
t5.state

## Passing data

In [ ]:
@dataclass
class Topic:
    
    name: str
           
    def set_presenter(self, presenter):
        self.presenter = presenter 
        
    def __post_init__(self):
        self.presenter = None
        self.machine = Machine(
            model=self, 
            states=['proposed', 'scheduled', 'completed', ], 
            initial='proposed',
            transitions=[
                {'trigger': 'schedule', 'source': 'proposed', 
                 'dest': 'scheduled', 
                 'before': 'set_presenter', },
                {'trigger': 'complete', 'source': 'scheduled', 'dest': 'completed'},        
            ],
            )  
t5 = Topic('Fluid Interfaces')

In [ ]:
t5.schedule()

In [ ]:
t5.schedule(presenter='Ken')

In [ ]:
t5.presenter

In [ ]:
t5.state

# Graphing

In [ ]:
!pip install transitions[diagrams]

In [ ]:
from transitions.extensions import GraphMachine
# Does not play well with dataclases!

In [ ]:

class Topic:
           
    def __init__(self, name):
        self.name = name
        self.presenter = None
        self.machine = GraphMachine(
            model=self, 
            states=['proposed', 'scheduled', 'completed', 'rejected'], 
            initial='proposed',
            transitions=[
                {'trigger': 'schedule', 'source': 'proposed', 'dest': 'scheduled'},
                {'trigger': 'complete', 'source': 'scheduled', 'dest': 'completed'},   
                {'trigger': 'reject', 'source': 'proposed', 'dest': 'rejected', }
            ])  
t6 = Topic('Java')

In [ ]:
t6.reject()

In [ ]:
t6.state

In [ ]:
graph = t6.get_graph()
graph

In [ ]:
graph.draw('states.svg', prog='dot')

In [ ]:
!xdg-open states.svg

In [ ]:
graph.draw(prog='dot')

# Task: add _repr_svg_ so that these can display directly in Notebook

## Logging

In [ ]:
import logging
logging.basicConfig(level=logging.INFO, filename='state-transitions.log',
                    format='%(asctime)s %(message)s')

@dataclass
class Topic:
    
    name: str
           
    def set_presenter(self, presenter):
        self.presenter = presenter 
        
    def __post_init__(self):
        self.presenter = None
        self.machine = Machine(
            model=self, 
            states=['proposed', 'scheduled', 'completed', ], 
            initial='proposed',
            transitions=[
                {'trigger': 'schedule', 'source': 'proposed', 
                 'dest': 'scheduled', 
                 'before': 'set_presenter', },
                {'trigger': 'complete', 'source': 'scheduled', 'dest': 'completed'},        
            ],
            )  
t5 = Topic('Fluid Interfaces')

In [ ]:
t5.schedule(presenter='Gary')

In [ ]:
t5.complete()

In [ ]:
cat state-transitions.log